In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='serif')
%matplotlib inline

import os, re, string, itertools

from recording import Recording
from neural_net import MultiLayerPerceptron
from auxiliary import *

from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

import keras 

In [ ]:
dh = DataHandler()

In [ ]:
def train_test_split(df,speaker_id):
    X_train = np.stack(df[df.speaker_id != speaker_id].feature_vec.tolist())
    y_train = df[df.speaker_id != speaker_id].emotion_label.tolist()
    X_test = np.stack(df[df.speaker_id == speaker_id].feature_vec.tolist())
    y_test = df[df.speaker_id == speaker_id].emotion_label.tolist()
    return X_train, X_test, y_train, y_test

# Condition 1
- Global features (averaged features over time)
- Train a single classifier

In [ ]:
df_1 = dh.build_feature_data(condition=1)

In [ ]:
df_1.head()

In [ ]:
def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}

In [ ]:
df_scores_1 = pd.DataFrame()
for i in range(5):
    scores, cms = [],[]
    for speaker_id in tqdm(sorted(set(df_1.speaker_id))):
        X_train, X_test, y_train, y_test = train_test_split(df_1,speaker_id)
        y_train = one_hot_encode(y_train, 7)

        scl = StandardScaler().fit(X_train)
        X_train = scl.transform(X_train)
        X_test = scl.transform(X_test)

        model = MultiLayerPerceptron([61,96,32,7])
        model.fit(X_train,y_train,epochs=100,val_data=(X_test,one_hot_encode(y_test,7)),class_weight=get_class_weights(df_1.emotion_label))
        pred = model.predict(X_test)
        scores.append(accuracy_score(y_test,pred))
        cms.append(confusion_matrix(y_test,model.predict(X_test)))
    df_scores_1['score_run_%i'%(i+1)] = scores
    df_scores_1['cm_run_%i'%(i+1)] = cms
    df_scores_1['speaker'] = sorted(set(df_1.speaker_id))

df_scores_1 = df_scores_1.set_index('speaker')
df_scores_1

# Condition 2
- Local features (features per phoneme)
- Train a single classifier

In [ ]:
def aggregate_score(predictions,IDs,dh):
    pred, labels = [],[]
    for x in np.unique(IDs):
        p = [predictions[i] for i in range(len(IDs)) if IDs[i] == x] 
        p = np.mean(p,axis=0)
        pred.append(np.argmax(p))
        labels.append(dh.emotion_from_ID(x))
    return accuracy_score(pred,labels), pred, labels

In [ ]:
df_2 = dh.build_feature_data(condition=2)

In [ ]:
df_scores_2 = pd.DataFrame()
for i in range(5):
    scores, cms = [],[]
    for speaker_id in tqdm(sorted(set(df_2.speaker_id))):
        X_train, X_test, y_train, y_test = train_test_split(df_2,speaker_id)
        y_train = one_hot_encode(y_train, 7)
        X_train = np.squeeze(X_train)
        X_test = np.squeeze(X_test)

        scl = StandardScaler().fit(X_train)
        X_train = scl.transform(X_train)
        X_test = scl.transform(X_test)

        model = MultiLayerPerceptron([61,96,32,7])
        model.fit(X_train,y_train,epochs=200,val_data=(X_test,one_hot_encode(y_test,7)),class_weight=get_class_weights(df_2.emotion_label))

        score,predictions,labels = aggregate_score(model.predict_proba(X_test).tolist(),
                                            df_2[df_2.speaker_id == speaker_id].ID.tolist(),
                                            dh)
        scores.append(score)
        cms.append(confusion_matrix(labels,predictions))
        keras.backend.clear_session()
    df_scores_2['score_run_%i'%(i+1)] = scores
    df_scores_2['cm_run_%i'%(i+1)] = cms
    df_scores_2['speaker'] = sorted(set(df_2.speaker_id))

keras.backend.clear_session()
    
df_scores_2 = df_scores_2.set_index('speaker')
df_scores_2

# Condition 3
- Local features (features per phoneme)
- Train one classifier per phoneme

In [ ]:
valid_phonemes = list(''.join([string.ascii_letters,string.digits,'@']))
df_3 = df_2[df_2.phoneme.isin(valid_phonemes)]
common_phonemes = {phoneme: count for phoneme,count in Counter(df_3.phoneme).most_common(6)}
df_3 = df_3[df_3.phoneme.isin(common_phonemes)]
common_phonemes

In [ ]:
df_scores_3 = pd.DataFrame()
phoneme_dict = {}
for i in range(5):
    scores, cms = [],[]
    phoneme_dict['run_%i'%i] = {}
    for speaker_id in tqdm(sorted(set(df_3.speaker_id))):
        df_train = df_3[df_3.speaker_id != speaker_id]
        df_test = df_3[df_3.speaker_id == speaker_id]
        scl = StandardScaler().fit(np.squeeze(np.stack(df_train.feature_vec.tolist())))

        model_dict = {}
        phoneme_dict['run_%i'%i]['speaker_%s'%speaker_id] = {}
        for phoneme in common_phonemes:
            model = MultiLayerPerceptron([61,96,32,7])

            tmp_train = df_train[df_train.phoneme == phoneme]
            X_train = np.squeeze(np.stack(tmp_train.feature_vec.tolist()))
            y_train = one_hot_encode(tmp_train.emotion_label.tolist(),7)
            tmp = df_test[df_test.phoneme == phoneme]
            X_test = np.squeeze(np.stack(tmp.feature_vec.tolist()))
            y_test = tmp.emotion_label.tolist()

            X_train = scl.transform(X_train)
            X_test = scl.transform(X_test)

            model.fit(X_train,y_train,epochs=200,val_data=(X_test,one_hot_encode(y_test,7)),class_weight=get_class_weights(tmp_train.emotion_label))
            model_dict[phoneme] = model
            
            # evaluate only this phoneme
            ph_pred = [model_dict[row.phoneme].predict_proba(scl.transform(row.feature_vec.reshape(1,-1))) 
                       for _,row in tmp.iterrows()]
            score,predictions,labels = aggregate_score(ph_pred,
                                                   tmp.ID.tolist(),
                                                   dh)
            phoneme_dict['run_%i'%i]['speaker_%s'%speaker_id][phoneme] = score

        predictions = [model_dict[row.phoneme].predict_proba(scl.transform(row.feature_vec.reshape(1,-1))) 
                       for _,row in df_test.iterrows()]

        score,predictions,labels = aggregate_score(predictions,
                                                   df_test.ID.tolist(),
                                                   dh)

        scores.append(score)
        cms.append(confusion_matrix(labels,predictions))
        keras.backend.clear_session()
    df_scores_3['score_run_%i'%(i+1)] = scores
    df_scores_3['cm_run_%i'%(i+1)] = cms
    df_scores_3['speaker'] = sorted(set(df_3.speaker_id))

df_scores_3 = df_scores_3.set_index('speaker')
df_scores_3

In [ ]:
df_scores_1[['score_run_%i'%(i+1) for i in range(5)]].to_csv('results/scores_1.csv')
# df_scores_2[['score_run_%i'%(i+1) for i in range(5)]].to_csv('results/scores_2.csv')
df_scores_3[['score_run_%i'%(i+1) for i in range(5)]].to_csv('results/scores_3.csv')

In [ ]:
import pickle
with open('results/cms_1.pkl','wb') as f:
    pickle.dump(df_scores_1[['cm_run_%i'%(i+1) for i in range(5)]].to_dict(),f)
# with open('results/cms_2.pkl','wb') as f:
#     pickle.dump(df_scores_2[['cm_run_%i'%(i+1) for i in range(5)]].to_dict(),f)
with open('results/cms_3.pkl','wb') as f:
    pickle.dump(df_scores_3[['cm_run_%i'%(i+1) for i in range(5)]].to_dict(),f)

## Performance by phoneme

In [ ]:
df = pd.DataFrame()
for i,k in enumerate(phoneme_dict):
    df = pd.concat([df,pd.DataFrame(phoneme_dict[k]).T])
df = df.reset_index()

In [ ]:
sns.set_context('talk')
plt.figure(figsize=(16,6),dpi=200)
sns.boxplot(x='Phoneme',y='Accuracy',whis=2,data=pd.melt(df,id_vars='index',var_name='Phoneme',value_name='Accuracy'))